# Lesson 1: Multi-Agent Conversation and Stand-up Comedy

## Setup

In [7]:
import os
import dotenv
from rich.pretty import pprint as print

dotenv.load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

llm_config = {
    "config_list": [
        {
            "model": "gpt-3.5-turbo",
            "api_key": openai_api_key
        }
    ]
}

## Define an AutoGen agent

In [8]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [9]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

"Sure, here's a light-hearted joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!"

In [10]:
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print(reply)

"Sure, I'd be happy to! Just let me know which joke you'd like me to repeat."

## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [11]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

**Note**: You might get a slightly different response (set of jokes) than what is shown in the video

In [12]:
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Got any new jokes for me today?

--------------------------------------------------------------------------------
joe (to cathy):

"Why did the scarecrow win an award?"

--------------------------------------------------------------------------------
cathy (to joe):

Why did the scarecrow win an award? Because he was outstanding in his field! Classic joke, Joe!

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (437e3e76-0d5b-4dcb-9b92-93a9e11fa83f): Maximum turns (2) reached


## Print some results

You can print out:

1. Chat history
2. Cost
3. Summary of the conversation

In [15]:
print(chat_result.chat_history)

[
│   {'content': "I'm Joe. Cathy, let's keep the jokes rolling.", 'role': 'assistant', 'name': 'joe'},
│   {'content': 'Hey Joe! Got any new jokes for me today?', 'role': 'user', 'name': 'cathy'},
│   {'content': '"Why did the scarecrow win an award?"', 'role': 'assistant', 'name': 'joe'},
│   {
│   │   'content': 'Why did the scarecrow win an award? Because he was outstanding in his field! Classic joke, Joe!',
│   │   'role': 'user',
│   │   'name': 'cathy'
│   }
]

In [16]:
print(chat_result.cost)

{
│   'usage_including_cached_inference': {
│   │   'total_cost': 0.00015099999999999998,
│   │   'gpt-3.5-turbo-0125': {
│   │   │   'cost': 0.00015099999999999998,
│   │   │   'prompt_tokens': 176,
│   │   │   'completion_tokens': 42,
│   │   │   'total_tokens': 218
│   │   }
│   },
│   'usage_excluding_cached_inference': {
│   │   'total_cost': 0.00015099999999999998,
│   │   'gpt-3.5-turbo-0125': {
│   │   │   'cost': 0.00015099999999999998,
│   │   │   'prompt_tokens': 176,
│   │   │   'completion_tokens': 42,
│   │   │   'total_tokens': 218
│   │   }
│   }
}

In [17]:
print(chat_result.summary)

'Why did the scarecrow win an award? Because he was outstanding in his field! Classic joke, Joe!'

## Get a better summary of the conversation

In [18]:
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! I see what you did there with the "rolling" joke setup. You must be on a roll today!

--------------------------------------------------------------------------------
joe (to cathy):


Thanks, Cathy! I guess you could say I'm "rolling" with the punches! Speaking of rolls, have you ever noticed how much we rely on technology these days?

--------------------------------------------------------------------------------
cathy (to joe):

Sorry, I didn't quite get it. Can you give me a bit more context?

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (1366255a-ed62-49c9-855b-677dc02feb57): Maximum turns (2) reached


In [19]:
print(chat_result.summary)

'Joe and Cathy are exchanging jokes and playful banter about rolling jokes and technology reliance.'

## Chat Termination

Chat can be terminated using a termination conditions.

In [20]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [21]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Glad you're up for some laughs. So, why did the scarecrow win an award?

Because he was outstanding in his field!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a good one, Cathy! Scarecrows are really setting the bar high. So, have you heard about the mathematician who's afraid of negative numbers? He'll stop at nothing to avoid them!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, that's a good one, Joe! That mathematician must really be positive about his decision to avoid negative numbers. Speaking of math jokes, did you hear about the angle that wanted to be friends with the circle? It just couldn't find a point where they intersected!

---------------------------------------------------------

In [22]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

The last joke we talked about was the one about the parallel lines having so much in common but never meeting.

--------------------------------------------------------------------------------
cathy (to joe):

Haha, that's right! Thanks for the laughs, Joe. Have a great day! See you next time!

--------------------------------------------------------------------------------
joe (to cathy):

Take care, Cathy! It was great joking around with you. Until next time!

--------------------------------------------------------------------------------
cathy (to joe):

Bye, Joe! Keep spreading those smiles!

--------------------------------------------------------------------------------
joe (to cathy):

Goodbye, Cathy! I'll do my best to keep the laughs coming!

--------------------------------------------------------------------------------
cathy 